# Sentiment Analysis of Movie Reviews using transformers and a Pretrained LLM

In this notebook, I fine-tune a pretrained LLM, DistilBERT base model (uncased), on the IMDB movie-review dataset in an effort to improve on the F1-test scores I got from the classifiers of scikit-learn. Once it is fine-tuned, I deploy the model to Hugging Face.

## Installs and Imports

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate
!pip install -U accelerate

In [2]:
import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import accelerate
import evaluate

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load the Data

In [4]:
ds_movies = load_dataset('imdb')

## Pretrained Model and Tokenizer

In [5]:
model_name = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize(dataset):
    """Tokenize the movie reviews of dataset."""
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_ds_movies = ds_movies.map(tokenize, batched=True)

## Train and Test

In [8]:
f1_score = evaluate.load('f1')

def compute_metrics(eval_pred):
    """Compute metrics for evaluation."""
    # raw outputs, actual labels
    logits, labels = eval_pred

    # predictions is the highest output probability
    predictions = np.argmax(logits, axis=-1)

    f1 = f1_score.compute(predictions=predictions, references=labels)
    return {'f1': f1['f1']}

In [9]:
training_args = TrainingArguments(
    output_dir='movie-review-classifier',
    eval_strategy='epoch',
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    push_to_hub=True,
    hub_model_id='derek-harnett/movie-review-classifier',
    hub_strategy='end'
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_movies['train'],
    eval_dataset=tokenized_ds_movies['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [12]:
trainer.evaluate()

{'eval_loss': 0.2742638885974884,
 'eval_f1': 0.9327129563350035,
 'eval_runtime': 467.3881,
 'eval_samples_per_second': 53.489,
 'eval_steps_per_second': 3.344,
 'epoch': 3.0}

## Push to Hugging Face

In [ ]:
trainer.push_to_hub(commit_message='train with full train set')